#Hossein Entezari Zarch
##810196419
#Artifical Intelligence
##Project3
Spring-2020

In the following part, The need packages are imported to be called in subsequent parts of the code.
CSV: CSV package is used in order to read from data and store it in python arrays.
NLTK: NLTK package is used to preprocess and clean text before it is put in algotithm for further works.

In [48]:
import csv
import nltk
nltk.download('wordnet')
from math import log
import random
from nltk.stem import PorterStemmer, SnowballStemmer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

The following function was defined in order to read the data from the file and store it in 2D array and then utlizes clean_text function to clean the text in each element of the array.

In [0]:
def read_csv(file_name):
    fields = []
    rows = []
    with open(file_name, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        fields = next(csv_reader)
        for row in csv_reader:
            rows.append(row)
        for row in rows:
            row[4] = clean_text(row[4])
            row[6] = clean_text(row[6])
    return fields, rows

def read_csv_test(file_name):
    fields = []
    rows = []
    with open(file_name, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        fields = next(csv_reader)
        for row in csv_reader:
            rows.append(row)
        for row in rows:
            row[1] = clean_text(row[1])
            row[4] = clean_text(row[4])
    return fields, rows

This function uses different methods available in NLTK package to tokenize, normalize and other preprocessing proccedures needed to be done on text.
Steps: At first, we tokenize the text with word_tokenize function available in nltk package gives us an array of wors in the input sentence, Then with calling isalpha() function on tokenized sentencs items we would get True if the word is an alphabetic otherwise, if the word is a punctuation of something elese we would get False on the output of the function, so in this line of code we remove the punctuations from the text, Then with calling lower() function we make all of the words to lowercase further, We know we need to remove the stopwords from the text for the proccdures we would perform on them so, we remove all of the words that are not in nltk.corpus.stopwords.words("english") which includes all of the english stopwords, At last we need to normalize the text the details of the process is available below:
In general there are two methods available to normalize the text "Stemming" and "Lemmatization".

Stemming: This method tries to normalized a word by reducing the word itself stem for instance, "automate", "automatic", "automation", "automations" will be reduced to "automat", it is proper to use when the stem of all forms of a word are the same, unless, lemmitization would be a better chioce.

Lemmatization: This method uses the wordnet database to perform the lemmatization.

Stemming VS Lemmatization: I think the choose of lemmatization is more convenient as this algorithm does not remove any important part of information we have about meaning of the word, on the other hand the Stemming algorithm reduce any word to its stem.

Need of lowercase the words: I think our algorithm for text processing is not dependant on the position of the word in the sentence, and most of the uppercase words represent the sentnece begins with that word which is not an crucial detail in out algorithm so, it would be better for us to lowercase the text to make similar words completely in an identical form.

When we experimented the result on the Stemming method instead of the Lemmatization method we observed that the results just changed a little that seems because of randomness and shuffling process in oversampling step and we didn't see any progress or detoriate in the performance of the algorithm.

Also we experimented without lowercase the text and we didn't see any meaningful change and at one experiment the total accracy decreased by 2 percent just.

In [0]:
def clean_text(text):
    tokens = nltk.word_tokenize(text)
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word.lower() for word in tokens]
    tokens = [word for word in tokens if not word in nltk.corpus.stopwords.words("english")]
    lemma = nltk.WordNetLemmatizer()
    tokens = [lemma.lemmatize(word, pos="v") for word in tokens]
    tokens = [lemma.lemmatize(word, pos="n") for word in tokens]
    # porter = PorterStemmer()
    # for i in range(len(tokens)):
    #     tokens[i] = porter.stem(tokens[i])
    return tokens

This function is implemented to seperate the validation data from training_data, the function iterates over the data rows and put 80 percent of them in the train_data and the 20 percent others in valid_data also it seperates different classes of data equally, also our proccdure was in a way that keeps the ratio of different classes of data items we have unchanged in order to be sure that the seperating the data doesn't cause any problem in having enough data of different classes both in training set and validation set.

In [0]:
def seperate_data(data):
    train_data = []
    valid_data = []
    dictNum = {}
    for row in data:
        if not row[2] in dictNum:
            dictNum[row[2]] = [1]
        else:
            dictNum[row[2]][0] += 1
    for row in data:
        if len(dictNum[row[2]]) < 2:
            dictNum[row[2]].append(1)
            train_data.append(row)
        elif dictNum[row[2]][1] < dictNum[row[2]][0] *(8/10):
            dictNum[row[2]][1] += 1
            train_data.append(row)
        else:
            valid_data.append(row)
    return train_data, valid_data

The following function, samples from the data with classes that their count is less than others and copy the sampled row in data then after perform a shuffling step it would return the final dataset.

This solution helps our algorithm to see data from different classes equally and not be biased on data given from one or two class with more amount of data in training set.

In [0]:
def over_sample(data):
    numClasses = [0, 0, 0]
    for row in data:
        if row[2] == 'BUSINESS':
            numClasses[0] += 1
        elif row[2] == 'TRAVEL':
            numClasses[1] += 1
        else:
            numClasses[2] += 1
    
    maxClass = max(numClasses)
    diff = maxClass - numClasses[0]
    newData = []
    while len(newData) < diff:
        s = random.choice(data)
        if s[2] == 'BUSINESS':
            newData.append(s)
    for row in newData:
        data.append(row)
    
    diff = maxClass - numClasses[1]
    newData = []
    while len(newData) < diff:
        s = random.choice(data)
        if s[2] == 'TRAVEL':
            newData.append(s)
    for row in newData:
        data.append(row)
    
    diff = maxClass - numClasses[2]
    newData = []
    while len(newData) < diff:
        s = random.choice(data)
        if s[2] == 'STYLE & BEAUTY':
            newData.append(s)
    for row in newData:
        data.append(row)
    random.shuffle(data)
    return data

Confusion Matrix is a performance measure for machine learning classification problems which generally includes num of true labled cases on each case and also num of false labled cases on each case depending on the false label the machine put on it.
In the following function you see how machine labels are distributed between classes of data.
and the result of the function on our model is:
<table>
    <tr>
        <th>actual label/predicted label</th>
        <th>BUSINESS</th>
        <th>TRAVEL</th>
        <th>STYLE & BEAUTY</th>
    </tr>
    <tr>
        <th>BUSINESS</th>
        <td>1598</td>
        <td>119</td>
        <td>61</td>
    </tr>
    <tr>
        <th>TRAVEL</th>
        <td>307</td>
        <td>1355</td>
        <td>116</td>
    </tr>
    <tr>
        <th>STYLE & BEAUTY</th>
        <td>185</td>
        <td>150</td>
        <td>1443</td>
    </tr>
<table>

In [0]:
def confusion_matrix(truth_labels):
    ans = {}
    for row in truth_labels:
        if not row[0] in ans:
            ans[row[0]] = {}
        else:
            if not row[1] in ans[row[0]]:
                ans[row[0]][row[1]] = 1
            else:
                ans[row[0]][row[1]] += 1
    return ans

Bayesian Rule : P(H|E)=P(E|H).P(H)/P(E)
The formulat above is the bayesian theorem, we are to get some detail on its parts.
-p(E): this term is the probability of the evidence occurance which is called  prior distribution, meaning that it represents our beliefs about truth of the parameters(here words we have in a short description).
-p(H|E): This part is called the posterior probability, more precisely it is the probabitliy of hypothesis given the observed evidence.
-p(E|H): This term is the probability of occuring E with given H and also it is named the likelihood, it indicates the compatibility of our evidence by the given hypothesis.
-P(E): This term is called evidence and it is the probability of occuring the evidence we have.

Proccedure of labeling: In our algorithm the main theorem we utilize is the bayesian theorem which makes it available for us to calculate p(c|X) when c represents a class and X represents the words in the sentence we are to label, the method is that to label a sentence containing words X, we calculate the p(c|X) for all available classes c and then label the c on sentence with the biggest value of p(c|X), according to the calculation approach we follow the formula as p(c|X) = p(X|c).p(c)/p(X) and as we know that, p(X) does not depend on c, so its calculation doesn't effect the comparison of p(c|X) values among different classes so we would omit this term from the formula, and also due to the situation of the problem, we know that, P(X) = p(x1).p(x2). ... as the p(x)s are independant to each other. so the final formulat we would use to compare between different classes on a sentence would be:
p(c|X) = p(x1|c).p(x2|c). ... .p(xn|c).p(c).

Calculate the value of p(xi|c):
-In our impelmentation, the value of p(xi|c) is the num of times we see the word xi in sentences with class c divided into number of all of words we have in sentences with class c.
-So, at learning step in train function, we store a triple array for each word we see in a dicionary that the first item stores num of times we saw word in the first class sentence and the second one and third one so on.
-The numWords triple array stores num of all words we see in each class sentences.


In [0]:
class Model:
    def __init__(self, file_name):
        self.data_fields, data = read_csv(file_name)
        data = over_sample(data)
        self.train_data, self.valid_data = seperate_data(data)
        self.test_data = None
    
    def train0(self):
        self.dictionary = {}
        self.numClasses = [0, 0, 0]
        self.numWords = [0, 0, 0]
        for row in self.train_data:
            if row[2] == 'BUSINESS':
                self.numClasses[0] += 1
            elif row[2] == 'TRAVEL':
                self.numClasses[1] += 1
            else:
                continue
            
            for word in row[6]:
                if not word in self.dictionary:
                    self.dictionary[word] = [0, 0, 0]
                if row[2] == 'BUSINESS':
                    self.dictionary[word][0] += 1
                    self.numWords[0] += 1
                elif row[2] == 'TRAVEL':
                    self.dictionary[word][1] += 1
                    self.numWords[1] += 1

    def train(self):
        self.dictionary = {}
        self.numClasses = [0, 0, 0]
        self.numWords = [0, 0, 0]
        for row in self.train_data:
            if row[2] == 'BUSINESS':
                self.numClasses[0] += 1
            elif row[2] == 'TRAVEL':
                self.numClasses[1] += 1
            elif row[2] == 'STYLE & BEAUTY':
                self.numClasses[2] += 1
            
            for word in row[6]:
                if not word in self.dictionary:
                    self.dictionary[word] = [0, 0, 0]
                if row[2] == 'BUSINESS':
                    self.dictionary[word][0] += 1
                    self.numWords[0] += 1
                elif row[2] == 'TRAVEL':
                    self.dictionary[word][1] += 1
                    self.numWords[1] += 1
                elif row[2] == 'STYLE & BEAUTY':
                    self.dictionary[word][2] += 1
                    self.numWords[2] += 1

    def label(self, datan, phase):
        if datan == "train":
            data = self.train_data
        elif datan == "valid":
            data = self.valid_data
        else:
            data = self.test_data
        
        labels = []
        for row in data:
            score1 = log(1)
            score2 = log(1)
            score3 = log(1)
            nameClasses = ['BUSINESS', 'TRAVEL', 'STYLE & BEAUTY']

            if datan == "test":
                sentence = row[4]
            else:
                sentence = row[6]

            for word in sentence:
                if not word in self.dictionary:
                    continue
                if self.dictionary[word][0] == 0:
                    score1 -= 10
                else:
                    score1 += log((self.dictionary[word][0])/self.numWords[0])
                if self.dictionary[word][1] == 0:
                    score2 -= 10
                else:
                    score2 += log((self.dictionary[word][1])/self.numWords[1])
                if phase == 1:
                    if self.dictionary[word][2] == 0:
                        score3 -= 10
                    else:
                        score3 += log((self.dictionary[word][2])/self.numWords[2])
            
            score1 += log((self.numClasses[0])/sum(self.numClasses))
            score2 += log((self.numClasses[1])/sum(self.numClasses))
            if phase == 1:
                score3 += log((self.numClasses[2])/sum(self.numClasses))

            if phase == 0:
                tmp = [score1, score2]
                labels.append(nameClasses[tmp.index(max(tmp))])
            else:
                tmp = [score1, score2, score3]
                labels.append(nameClasses[tmp.index(max(tmp))])
        return labels
            
    def scores(self, data, phase):
        def count_correct_labels(truth_labels, label):
            count = 0
            for row in truth_labels:
                if row[0] == row[1] and row[0] == label:
                    count +=1
            return count
        
        def count_detected_labels(truth_labels, label, phase):
            count = 0
            for row in truth_labels:
                if phase==0 and row[0] == 'STYLE & BEAUTY':
                    continue
                if row[1] == label:
                    count += 1
            return count   
        
        def count_labels(truth_labels, label):
            count = 0
            for row in truth_labels:
                if row[0] == label:
                    count += 1
            return count
        
        def count_trues(truth_labels, phase):
            count = 0
            all = 0
            for row in truth_labels:
                if phase == 0 and row[0] == 'STYLE & BEAUTY':
                    continue
                all += 1
                if row[0] == row[1]:
                    count += 1
            return count, all
        
        labels = self.label(data, phase)

        if data == "train":
            data = self.train_data
        elif data == "valid":
            data = self.valid_data
        else:
            data = self.test_data
        

        recalls = {'BUSINESS':0, 'TRAVEL':0, 'STYLE & BEAUTY':0}
        precisions = {'BUSINESS':0, 'TRAVEL':0, 'STYLE & BEAUTY':0}
        accuracys = 0

        truth_labels = []
        for i in range(len(data)):
            tmp = []
            tmp.append(data[i][2])
            tmp.append(labels[i])
            truth_labels.append(tmp)
        confusion = confusion_matrix(truth_labels)
        print("confusion matrix ")
        print(confusion)
        recalls['BUSINESS'] = count_correct_labels(truth_labels, 'BUSINESS')/count_labels(truth_labels, 'BUSINESS')
        precisions['BUSINESS'] = count_correct_labels(truth_labels, 'BUSINESS')/count_detected_labels(truth_labels, 'BUSINESS', phase)
        recalls['TRAVEL'] = count_correct_labels(truth_labels, 'TRAVEL')/count_labels(truth_labels, 'TRAVEL')
        precisions['TRAVEL'] = count_correct_labels(truth_labels, 'TRAVEL')/count_detected_labels(truth_labels, 'TRAVEL', phase)
        
        if phase==1:
            recalls['STYLE & BEAUTY'] = count_correct_labels(truth_labels, 'STYLE & BEAUTY')/count_labels(truth_labels, 'STYLE & BEAUTY')
            precisions['STYLE & BEAUTY'] = count_correct_labels(truth_labels, 'STYLE & BEAUTY')/count_detected_labels(truth_labels, 'STYLE & BEAUTY', phase)

        trues, alll = count_trues(truth_labels, phase)
        accuracys = trues/alll
        return recalls, precisions, accuracys

    def test(self, file_name):
        data_fields, self.test_data = read_csv_test(file_name)
        labels = self.label("test", 1)
        with open('output.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['index', 'category'])
            for i,row in enumerate(labels):
                writer.writerow([i, row])


Result Tables
<table>
    <tr>
        <th>Phase1</th>
        <th>Travel</th>
        <th>Business</th>
    </tr>
    <tr>
        <th>Recall</th>
        <td>0.80</td>
        <td>0.90</td>
    </tr>
    <tr>
        <th>Precision</th>
        <td>0.89</td>
        <td>0.82</td>
    </tr>
    <tr>
        <th>Accuracy</th>
        <td></td>
        <td>0.85</td>
    </tr>
</table>
</br>
<table>
    <tr>
        <th>Phase2</th>
        <th>Travel</th>
        <th>Business</th>
        <th>Style & Beauty</th>
    </tr>
    <tr>
        <th>Recall</th>
        <td>0.76</td>
        <td>0.90</td>
        <td>0.81</td>
    </tr>
    <tr>
        <th>Precision</th>
        <td>0.83</td>
        <td>0.76</td>
        <td>0.89</td>
    </tr>
    <tr>
        <th>Accuracy</th>
        <td></td>
        <td>0.82</td>
        <td></td>
    </tr>
</table>

##What happens if we just have a high amount of precision?

Suppose that you have system that diagnoses a disease and just 1 percent of people who are experimented are sick, so if your machine label every person as healthy, so your precision for healthy people would be 0.99 but your machine does nothing at all, so we cannot take the precision value as a measurement for evaluation of our machine.

##TF-IDF
This is a measurment to identify the importance of a word to a document in a corpus or collection.

Computation: TermFrequency * InverseDocumentFrequency

TermFrequency: tf(t,d) = num of occurance of term t in document d/num of all words in document d

InverseDocumentFrequency: idf(t,d)=log(num of occurance of the term t/num of the documents containing term t)

the tf(t,d) specifies the how common a word is in the document, and the idf(t,d) specifies how unique or rare a word is among all the corpus

So through this approach we would identify the most important words in every document and prune the ones that are totally much frequent and not important in the corpus.

In order to use this method in our bayesian approach, I think we can multiplicate the second term of the tf-idf to the probability we compute for each word, and the term would be log(num of sentences have the word/num of classes have the word) and this term would be a measurement how the word is distributed along different classes.

##What happens if you see a word you haven't seen in any other class?

According to our method of computation, if the num of occurance of the word in a class is zero it means log of its probability would be -inf and it cannot be stored well, so we put -10 instead which causes a much level of decrease for the chance of the class to be chosen.

In [55]:
model = Model("data.csv")
model.train()
a, b, c = model.scores("valid", 1)
print("Recall", a)
print("Precision", b)
print("Accuracy", c)
model.test("test.csv")

confusion matrix 
{'BUSINESS': {'BUSINESS': 1583, 'TRAVEL': 118, 'STYLE & BEAUTY': 77}, 'STYLE & BEAUTY': {'STYLE & BEAUTY': 1455, 'TRAVEL': 140, 'BUSINESS': 183}, 'TRAVEL': {'BUSINESS': 316, 'TRAVEL': 1334, 'STYLE & BEAUTY': 128}}
Recall {'BUSINESS': 0.8903878583473862, 'TRAVEL': 0.7504215851602024, 'STYLE & BEAUTY': 0.8184373243395165}
Precision {'BUSINESS': 0.7604416706673067, 'TRAVEL': 0.8380414312617702, 'STYLE & BEAUTY': 0.876580373269115}
Accuracy 0.8197489226157018
